In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
import pandas as pd
from io import StringIO

load_dotenv()

/tmp/ipykernel_20025/1868981330.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


True

In [3]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
)
import pandas as pd
import json


In [10]:

# TODO: add functionaility to use other libs
system_template_1 = """
You are very powerful code generation assistant who specalises in generating code to visualise graphs. You should always
validate the code, but you are no good at validating this code yourself so you must use the CodeValidationTool.
Here are some assumptions you should always follow:

- Should always plot using python and with the plotly library
- The df is stored under df.csv
- Save the figure via `pio.write_image(fig, 'figure.png')` and import plotly.io as pio at the top of the script 
- Provide a brief description of what the plot is about in context of the data

Here is some metadata about the data:
{metadata_json}
"""

code_generation_agent_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_template_1),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        ("user", "{user_input}"),
        # MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

code_generation_agent_prompt.format_messages(
    user_input="I want to plot a graph of the data",
    metadata_json='sd'
)



[SystemMessage(content="\nYou are very powerful code generation assistant who specalises in generating code to visualise graphs. You should always\nvalidate the code, but you are no good at validating this code yourself so you must use the CodeValidationTool.\nHere are some assumptions you should always follow:\n\n- Should always plot using python and with the plotly library\n- The df is stored under df.csv\n- Save the figure via `pio.write_image(fig, 'figure.png')` and import plotly.io as pio at the top of the script \n- Provide a brief description of what the plot is about in context of the data\n\nHere is some metadata about the data:\nsd\n"),
 HumanMessage(content='I want to plot a graph of the data')]

In [4]:
validation_llm_prompt = ChatPromptTemplate.from_messages(
    [
        {
            "role": "system",
            "content": """You are a highly skilled validation agent who will be examining plots genreated from pythons plotly library.
            You should validate if the plot is legible any anything is majorly wrong with it. such examples could be:
            - The plot is not legible
            - axis labels read to read
            - Something is obviously wrong with the plot
            If you find anything wrong with the plot, please provide feedback on how to fix it in plain english. If the plot is sufficently legible, please return "The plot is legible". 
            If not please say in the output to regenerate the code with the changes and run the code through this validation tool before returning code to user.
            """,
        }
    ]
)    

NotImplementedError: Unsupported message type: <class 'dict'>

In [2]:
from langchain.tools import E2BDataAnalysisTool
import json

In [20]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [21]:
df = pd.read_csv("jobs_in_data.csv")


def extract_metadata(df: pd.DataFrame, col_limit=150):
    if df.shape[1] > col_limit:
        raise ValueError(f"Dataframe exceeds col limit of {col_limit}")

    df = df.copy()
    metadata_dict = {
        "column_names": df.columns.tolist(),
        "data_dimensions": df.shape,
        "data_types_per_column": df.dtypes.apply(lambda x: x.name).to_dict(),
        "statistical_summary": df.describe().to_dict(),
        "missing_values_per_column": df.isnull().sum().to_dict(),
    }
    return json.dumps(metadata_dict)


df = pd.read_csv("jobs_in_data.csv")
metadata_json = extract_metadata(df)

In [22]:
from parsers import code_generation_output_parser

In [23]:
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts.chat import ChatPromptTemplate

In [24]:
system_template_1 = "You are a coding assistant designed to create code to plot graphs in python in plotly.You should return a breif description and the code to plot the graph"
system_template_2 = "Here is some metadata about the data: {metadata_json}"
human_message_template = "{human_message} /n {format_instructions}"

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_template_1),
        ("system", system_template_2),
        ("human", human_message_template),
    ]
)

messages = chat_prompt.format_messages(
    metadata_json=metadata_json,
    human_message="Make a plot of the average salary of every job.",
    format_instructions=code_generation_output_parser.get_format_instructions(),
)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=OPENAI_API_KEY, temperature=0)

In [25]:
print(code_generation_output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"code": {"title": "Code", "description": "code to generate the plots", "type": "string"}, "description": {"title": "Description", "description": "brief description of the code and plots", "type": "string"}}, "required": ["code", "description"]}
```


In [27]:
response = llm.invoke(messages)

In [28]:
print(response.content)

To plot the average salary of every job, you can use the following code:

```python
import plotly.express as px
import pandas as pd

# Assuming your data is stored in a pandas DataFrame called 'df'
average_salary = df.groupby('job_title')['salary'].mean().reset_index()

# Plotting the average salary for each job
fig = px.bar(average_salary, x='job_title', y='salary', title='Average Salary by Job')
fig.show()
```

This code calculates the average salary for each job title using the `groupby` function in pandas. Then, it uses the `px.bar` function from Plotly Express to create a bar chart with the job titles on the x-axis and the average salary on the y-axis.

The resulting plot will show the average salary for each job title.

Here is the JSON instance conforming to the provided schema:

```json
{
  "code": "import plotly.express as px\nimport pandas as pd\n\n# Assuming your data is stored in a pandas DataFrame called 'df'\naverage_salary = df.groupby('job_title')['salary'].mean().reset

In [29]:
code_generation_output_parser.parse(response.content)

CodeAndDescription(code="import plotly.express as px\nimport pandas as pd\n\n# Assuming your data is stored in a pandas DataFrame called 'df'\naverage_salary = df.groupby('job_title')['salary'].mean().reset_index()\n\n# Plotting the average salary for each job\nfig = px.bar(average_salary, x='job_title', y='salary', title='Average Salary by Job')\nfig.show()", description='This code calculates the average salary for each job title and plots it using a bar chart.')

## Validation Tool

In [70]:
class NamedStringIO(StringIO):
    """Custom StringIO class with name attribute to circumvent e2b .upload_file()
    method needing file to be saved locally.
    """

    def __init__(self, *args, **kwargs):
        self.name = kwargs.pop("name", "untitled")
        super().__init__(*args, **kwargs)


def upload_df_to_sandbox(df, sandbox):
    # TODO: add functionailty to upload multiple df's
    # Convert DataFrame to CSV format in memory
    csv_buffer = NamedStringIO(name="df.csv")
    df.to_csv(csv_buffer, index=False)
    csv_buffer.seek(0)  # Go to the start of the StringIO object

    # Upload CSV to sandbox
    sandbox.upload_file(csv_buffer)

In [92]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from typing import Optional, Type
import base64
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from e2b import Sandbox
import logging
import requests

Logger = logging.Logger(__name__)


class ValidationToolInput(BaseModel):
    # TODO: figure out right definnition for this so agent can recognise
    code: str = Field(description="python code to generate plots")


# TODO: add logging
# TODO: add pydantic validation as method here
class CustomValidationTool(BaseTool):
    name = "CodeValidationTool"
    description = "useful for when you have code that generates a plot and you want to validate it"
    args_schema: Type[BaseModel] = ValidationToolInput

    binary = "python3"
    filepath = "/index.py"
    df = pd.DataFrame()

    def _run(
        self, code: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""

        image_in_bytes = self._execute_code(code)
        return self._validate_image(image_in_bytes)

    def _execute_code(self, code: str) -> str:
        # TODO: handle case where this fails
        sandbox = Sandbox(
            template="Python3-DataAnalysis",
        )
        upload_df_to_sandbox(self.df, sandbox)
        # TODO: figure this out adds to much time make docker image?
        package_install = sandbox.process.start("pip install -U kaleido")
        package_install.wait()

        sandbox.filesystem.write(self.filepath, code)

        output = sandbox.process.start_and_wait(cmd=f"{self.binary} {self.filepath}")
        print(output.stdout)
        print(output.stderr)
        print('---')
        
        Logger.info(f"E2B Sandbox stdout: {output.stdout}")
        Logger.info(f"E2B Sandbox stderr: {output.stderr}")

        # TODO: ensure this is only defined once when doing prompting on save file name
        image_in_bytes = sandbox.download_file("/home/user/figure.png")

        sandbox.close()



        return image_in_bytes

    def _validate_image(self, image_in_bytes: bytes) -> str:

        base64_string = base64.b64encode(image_in_bytes).decode("utf-8")

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {OPENAI_API_KEY}",
        }

        payload = {
            "model": "gpt-4-vision-preview",
            "messages": [
                {
                    "role": "system",
                    "content": """You are a highly skilled validation agent who will be examining plots genreated from pythons plotly library.
            You should validate if the plot is legible any anything is majorly wrong with it. such examples could be:
            - The plot is not legible
            - axis labels read to read
            - Something is obviously wrong with the plot
            If you find anything wrong with the plot, please provide feedback on how to fix it in plain english. If the plot is sufficently legible, please return "The plot is legible".
            """,
                },
                # TODO: insert the code description in here?
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "Please validate this plot I made using the plotly library",
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_string}"
                            },
                        },
                    ],
                },
            ],
            "max_tokens": 300,
        }

        response = requests.post(
            "https://api.openai.com/v1/chat/completions", headers=headers, json=payload
        )
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]

    # TODO: figure out how to define this
    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [32]:
code = """
import pandas as pd
import plotly.express as px
import plotly.io as pio

df = pd.read_csv('jobs_in_data.csv')
print(df.head())
print(df.columns)

# Group the data by job title and calculate the average salary
average_salary = df.groupby('job_title')['salary'].mean().reset_index()
# Plot the data
fig = px.bar(average_salary, x='job_title', y='salary', title='Average Salary by Job Title')
# make label text larger
fig.update_layout(
    font=dict(
        size=40
    )
)

pio.write_image(fig, 'figure.png')
print('done')
"""

In [33]:
validation_tool = CustomValidationTool(df=df)
output = validation_tool.run(code)

In [34]:
output

'The plot is not legible due to several issues:\n\n1. The axis labels are sideways, which makes them hard to read.\n2. The axis labels are overlapping with each other, which further impedes legibility.\n3. The title of the plot is cut off; only "Average Salary by Job T" is visible.\n4. The values on the y-axis (Values) and the actual bars of the plot are obscured by the x-axis labels.\n\nTo fix these issues:\n\n- Rotate the x-axis labels so they are slanted or horizontal to prevent overlap and to make them easier to read.\n- Ensure the full title of the plot is visible.\n- Adjust the layout so that there is a clear separation between the axis labels and the data in the graph, making all elements clearly visible.'

In [35]:
# plot this
from IPython.display import Image

Image(file_in_bytes)

NameError: name 'file_in_bytes' is not defined

## Agent

In [83]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI

In [98]:
df = pd.read_csv("jobs_in_data.csv")
metadata_json = extract_metadata(df)

In [93]:
tool = CustomValidationTool(df=df)
prompt = hub.pull("hwchase17/openai-functions-agent")
tools = [tool]

In [94]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=OPENAI_API_KEY, temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)

# TODO: turn handle_parsing_errors off whilst developing
# TODO: think about config for testing and acc deployment?
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=2,
)

In [105]:
example_input = (
    """
Generate the python code using the plotly library to make a plot of the average salary of every job
make sure at the end of the code you save the figure via `pio.write_image(fig, 'figure.png')` and
import plotly.io as pio at the top of the script also. Also assume that the df is stored under df.csv.
"""
    + metadata_json
    + """\nOnce this code is generated validate this code"""
)

In [106]:
agent_executor.invoke(
    {
        "input": example_input
    }
)



> Entering new AgentExecutor chain...
Could not parse tool input: {'arguments': "import plotly.express as px\nimport plotly.io as pio\nimport pandas as pd\n\n# Read the data from df.csv\ndf = pd.read_csv('df.csv')\n\n# Calculate the average salary for each job\naverage_salary = df.groupby('job_title')['salary'].mean().reset_index()\n\n# Create the plot\nfig = px.bar(average_salary, x='job_title', y='salary', title='Average Salary by Job')\nfig.update_layout(xaxis_title='Job Title', yaxis_title='Average Salary')\n\n# Save the figure\npio.write_image(fig, 'figure.png')", 'name': 'python'} because the `arguments` is not valid JSON.Invalid or incomplete responseCould not parse tool input: {'arguments': "import plotly.express as px\nimport plotly.io as pio\nimport pandas as pd\n\n# Read the data from df.csv\ndf = pd.read_csv('df.csv')\n\n# Calculate the average salary for each job\naverage_salary = df.groupby('job_title')['salary'].mean().reset_index()\n\n# Create the plot\nfig = px.bar(a

{'input': '\nGenerate the python code using the plotly library to make a plot of the average salary of every job\nmake sure at the end of the code you save the figure via `pio.write_image(fig, \'figure.png\')` and\nimport plotly.io as pio at the top of the script also. Also assume that the df is stored under df.csv.\n{"column_names": ["work_year", "job_title", "job_category", "salary_currency", "salary", "salary_in_usd", "employee_residence", "experience_level", "employment_type", "work_setting", "company_location", "company_size"], "data_dimensions": [9355, 12], "data_types_per_column": {"work_year": "int64", "job_title": "object", "job_category": "object", "salary_currency": "object", "salary": "int64", "salary_in_usd": "int64", "employee_residence": "object", "experience_level": "object", "employment_type": "object", "work_setting": "object", "company_location": "object", "company_size": "object"}, "statistical_summary": {"work_year": {"count": 9355.0, "mean": 2022.7604489577766, "s

In [79]:
print(
"import plotly.express as px\nimport plotly.io as pio\nimport pandas as pd\n\n# Read the data from the CSV file\ndf = pd.read_csv('df.csv')\n\n# Calculate the average salary for each job\navg_salary = df.groupby('job')['salary'].mean().reset_index()\n\n# Create the plot\nfig = px.bar(avg_salary, x='job', y='salary', title='Average Salary by Job')\n\n# Save the figure\npio.write_image(fig, 'figure.png')")

import plotly.express as px
import plotly.io as pio
import pandas as pd

# Read the data from the CSV file
df = pd.read_csv('df.csv')

# Calculate the average salary for each job
avg_salary = df.groupby('job')['salary'].mean().reset_index()

# Create the plot
fig = px.bar(avg_salary, x='job', y='salary', title='Average Salary by Job')

# Save the figure
pio.write_image(fig, 'figure.png')


In [68]:
df

,Job,Salary
0,Engineer,50000
1,Manager,70000
2,Analyst,60000
3,Developer,55000


In [91]:
code = """ 
import plotly.express as px
import plotly.io as pio
import pandas as pd

# Read the data from the CSV file
df = pd.read_csv('df.csv')

# Calculate the average salary for each job
avg_salary = df.groupby('job')['salary'].mean().reset_index()

# Create the plot
fig = px.bar(avg_salary, x='job', y='salary', title='Average Salary by Job')

# Save the figure
pio.write_image(fig, 'figure.png')
"""

tool.run(code)

Exception: Failed to download file '/home/user/figure.png'. Not Found File not found: open /home/user/figure.png: no such file or directory


WebSocket received error while receiving messages: no close frame received or sent


In [65]:
import plotly.express as px
import plotly.io as pio

# # create a dataframe with job and salary
# data = {
#     'Job': ['Engineer', 'Manager', 'Analyst', 'Developer'],
#     'Salary': [50000, 70000, 60000, 55000]
# }
# df = pd.DataFrame(data)

# # create a bar plot
# fig = px.bar(df, x='Job', y='Salary', title='Average Salary by Job')

# save the figure
# pio.write_image(fig, 'figure.png')

In [69]:
df = pd.read_csv("jobs_in_data.csv")